In [1]:
import matplotlib.pyplot as plt
import scipy.stats as stats
from pyddm import Model
from pyddm import Sample
import pyddm as ddm
import numpy as np
import pandas as pd
import random
import os
import copy

## Fit M1: basic model

In [2]:
from pyddm import InitialCondition,Fittable, Fitted
from pyddm.models import Drift, Bound, LossRobustBIC, NoiseConstant, BoundConstant, OverlayChain, OverlayNonDecision, OverlayPoissonMixture
from pyddm.functions import fit_adjust_model

class ICPointSideBiasRatioC(InitialCondition):
    name = "A side-biased starting point expressed as a proportion of the distance between the bounds."
    required_parameters = ["x00", "x01"]
    required_conditions = ["present", "condition"]
    def get_IC(self, x, dx, conditions):
        x0 = (conditions['condition']*self.x01 + (1-conditions['condition'])*self.x00)/2 + .5 #rescale to between 0 and 1

        # Bias > .5 for left side correct, bias < .5 for right side correct.
        # On original scale, positive bias for left, negative for right
        if not conditions['present']==1:
            x0 = 1-x0
        shift_i = int((len(x)-1)*x0)
        assert shift_i >= 0 and shift_i < len(x), "Invalid initial conditions"
        pdf = np.zeros(len(x))
        pdf[shift_i] = 1. # Initial condition at x=x0*2*B.
        return pdf
    
class DriftPresentC(ddm.models.Drift):
    name = "Drift depends on target presence"
    required_parameters = ["driftpresent0","driftabsent0", "driftpresent1","driftabsent1"] # <-- Parameters we want to include in the model
    required_conditions = ["present", "condition"] # <-- Task parameters ("conditions"). Should be the same name as in the sample.
    # We must always define the get_drift function, which is used to compute the instantaneous value of drift.
    def get_drift(self, conditions, **kwargs):
        driftpresent = [self.driftpresent0, self.driftpresent1][int(conditions['condition'])]
        driftabsent = [self.driftabsent0, self.driftabsent1][int(conditions['condition'])]
        # so that driftabsent values are normally negative
        return driftpresent * conditions['present'] - driftabsent * (1-conditions['present'])

class BoundC(Bound):
    name = "Boundary by condition"
    required_parameters = ["B0", "B1"]
    required_conditions = ["condition"]
    def get_bound(self, t, conditions, **kwargs):
        return conditions['condition']*self.B1 + (1-conditions['condition'])*self.B0
                                                  
class OverlayNonDecisionC(OverlayNonDecision):
    name = "Non-decision time by condition"
    required_parameters = ["nondectime0", "nondectime1"]
    required_conditions = ["condition"] # Side coded as 0=L or 1=R
    def get_nondecision_time(self, conditions):
        return conditions['condition']*self.nondectime0 + (1-conditions['condition'])*self.nondectime1



In [33]:
m1c_spec = Model(name='Basic model drift varies as a function of target presence',
                 drift=DriftPresentC(driftabsent0=Fittable(minval=-10, maxval=0),
                                     driftpresent0=Fittable(minval=0, maxval=10),
                                     driftabsent1=Fittable(minval=-10, maxval=0),
                                     driftpresent1=Fittable(minval=0, maxval=10)),
                 noise=NoiseConstant(noise=1),
                 IC=ICPointSideBiasRatioC(x00=Fittable(minval=-1, maxval=1),
                                         x01=Fittable(minval=-1, maxval=1)),
                 bound=BoundC(B0=Fittable(minval=0.1, maxval=2.5),
                             B1 = Fittable(minval=0.1, maxval=2.5)),
                 overlay=OverlayChain(overlays=[OverlayNonDecisionC(nondectime0=Fittable(minval=0, maxval=1),
                                                                   nondectime1=Fittable(minval=0,maxval=1)),
                                                OverlayPoissonMixture(pmixturecoef=.02,
                                                                      rate=1)]),
                 dx=.01, dt=.1, T_dur=5)

def fit_m1c(df,data_label):
    
    df = df.reset_index();
    subj_id = df.subj_id[0]
    
    sample = Sample.from_pandas_dataframe(df, rt_column_name="rt", correct_column_name="correct")
    fit_model = fit_adjust_model(sample=sample, model=m1c_spec, verbose=False, lossfunction = LossRobustBIC)
    summary_df = pd.DataFrame.from_dict({
        'subj_id': [subj_id],
        'driftabsent0': [float(fit_model.parameters()['drift']['driftabsent0'])],
        'driftpresent0': [float(fit_model.parameters()['drift']['driftpresent0'])],
        'noise': [float(fit_model.parameters()['noise']['noise'])],
        'bound0': [float(fit_model.parameters()['bound']['B0'])],
        'IC0': [float(fit_model.parameters()['IC']['x00'])],
        'nondec0': [float(fit_model.parameters()['overlay']['nondectime0'])],
        'driftabsent1': [float(fit_model.parameters()['drift']['driftabsent1'])],
        'driftpresent1': [float(fit_model.parameters()['drift']['driftpresent1'])],
        'bound1': [float(fit_model.parameters()['bound']['B1'])],
        'IC1': [float(fit_model.parameters()['IC']['x01'])],
        'nondec1': [float(fit_model.parameters()['overlay']['nondectime1'])],
        'BIC': [fit_model.fitresult.value()]
    })
    
    if not os.path.exists("model_fits/model_m1c/data_"+data_label):
                       os.makedirs("model_fits/model_m1c/data_"+data_label)
                       
    with open("model_fits/model_m1c/data_"+data_label+"/subj"+str(subj_id)+".txt", "w") as f:
        f.write(repr(fit_model))
    
    return(summary_df)

    
loaded = pd.read_csv("data/E2.csv") 
loaded['present'] = loaded['present'].replace(-1,0)
loaded['condition'] = loaded['condition_is_congruent'].replace(-1,0)

summary_list = []
for name, group in loaded.groupby('subj_id'):
    print(name)
    a= fit_m1c(group.reset_index(),'E2')
    summary_list.append(a)
summary_df = pd.concat(summary_list)
summary_df.to_csv("model_fits/model_m1c/data_E2/all_subjects.csv")

1


Info: Params [ 0.29195065 -1.52399571  0.         -2.22402815  0.69780503  0.6512712
  0.35838543  0.41673261  0.62052341  0.51563228] gave 116.81331269994428


2


Info: Params [ 0.         -1.1555448   0.         -1.52428748  0.91918736  0.91921042
  0.23828316  0.36157191  0.83285918  0.95784483] gave 185.54241348545816


3


...

297


Info: Params [ 0.36501626 -0.95693077  0.34062084 -0.92512083  0.95176253  1.00668778
  0.10730013 -0.01887038  0.97722111  0.9712809 ] gave 210.78915136576458


298


Info: Params [ 0.6809514  -0.71787613  0.48439048 -0.16337871  1.50157522  1.15701113
  0.23102767  0.20928026  0.93170688  0.96196749] gave 252.921480136647


PermissionError: [Errno 13] Permission denied: 'model_fits/model_m1c/data_E2/all_subjects.csv'

## Fit M2: boundaries can collapse

In [3]:
from pyddm import InitialCondition,Fittable, Fitted
from pyddm.models import Drift, Bound, LossRobustBIC, NoiseConstant, OverlayChain, OverlayNonDecision, OverlayPoissonMixture, Noise
from pyddm.functions import fit_adjust_model
    
class BoundCollapseC(Bound):
    name = "Boundary for the collapsing boundary model"
    required_parameters = ["B0", "uppercollapse0", "lowercollapse0", "B1", "uppercollapse1", "lowercollapse1"]
    required_conditions = ["condition"]
    def get_bound(self, t, conditions, **kwargs):
        B = [self.B0, self.B1][int(conditions['condition'])]
        uppercollapse = [self.uppercollapse0, self.uppercollapse1][int(conditions['condition'])]
        lowercollapse = [self.lowercollapse0, self.lowercollapse1][int(conditions['condition'])]

        return max(0.01, B + t*(uppercollapse-lowercollapse)/2)
    
class DriftPresentCollapseC(ddm.models.Drift):
    name = "Drift depends on target presence and collapses"
    required_parameters = ["driftpresent0", "uppercollapse0", "lowercollapse0", "driftpresent1", "uppercollapse1", "lowercollapse1"] # <-- Parameters we want to include in the model
    required_conditions = ["present", "condition"] # <-- Task parameters ("conditions"). Should be the same name as in the sample.

    # We must always define the get_drift function, which is used to compute the instantaneous value of drift.
    def get_drift(self, t, conditions, **kwargs):
        
        uppercollapse = [self.uppercollapse0, self.uppercollapse1][int(conditions['condition'])]
        lowercollapse = [self.lowercollapse0, self.lowercollapse1][int(conditions['condition'])] 
        driftpresent = [self.driftpresent0, self.driftpresent1][int(conditions['condition'])]
        
        # so that driftabsent values are normally negative
        return conditions['present'] * driftpresent - (uppercollapse+lowercollapse)/2 + \
    (1-conditions['present']) *(0+ (uppercollapse+lowercollapse)/2) 





In [42]:
uppercollapse0 = Fittable(minval=-5, maxval=0)
lowercollapse0 = Fittable(minval=0, maxval=5)
uppercollapse1 = Fittable(minval=-5, maxval=0)
lowercollapse1 = Fittable(minval=0, maxval=5)

m2c_spec = Model(name='Absence drift is zero boundaries can collapse',
                 drift=DriftPresentCollapseC(driftpresent0=Fittable(minval=0, maxval=10),
                                             driftpresent1 = Fittable(minval=0, maxval=10),
                                   uppercollapse0=uppercollapse0,
                                   lowercollapse0=lowercollapse0,
                                   uppercollapse1=uppercollapse1,
                                   lowercollapse1=lowercollapse1),
                 noise=NoiseConstant(noise=1),
                 IC=ICPointSideBiasRatioC(x00=Fittable(minval=-1, maxval=1),
                                         x01=Fittable(minval=-1, maxval=1)),
                 bound=BoundCollapseC(B0=Fittable(minval=0.1, maxval=2.5),
                             B1 = Fittable(minval=0.1, maxval=2.5),
                                   uppercollapse0=uppercollapse0,
                                   lowercollapse0=lowercollapse0,
                                   uppercollapse1=uppercollapse1,
                                   lowercollapse1=lowercollapse1),
                 overlay=OverlayChain(overlays=[OverlayNonDecisionC(nondectime0=Fittable(minval=0, maxval=1),
                                                                   nondectime1=Fittable(minval=0,maxval=1)),
                                                OverlayPoissonMixture(pmixturecoef=.02,
                                                                      rate=1)]),
                 dx=.01, dt=.1, T_dur=5)


def fit_m2c(df,data_label):
    
    df = df.reset_index();
    subj_id = df.subj_id[0]
    
    sample = Sample.from_pandas_dataframe(df, rt_column_name="rt", correct_column_name="correct")
    fit_model = fit_adjust_model(sample=sample, model=m2c_spec, verbose=False, lossfunction = LossRobustBIC)
    summary_df = pd.DataFrame.from_dict({
        'subj_id': [subj_id],
        'driftpresent0': [float(fit_model.parameters()['drift']['driftpresent0'])],
        'noise': [float(fit_model.parameters()['noise']['noise'])],
        'bound0': [float(fit_model.parameters()['bound']['B0'])],
        'uppercollapse0': [float(fit_model.parameters()['drift']['uppercollapse0'])],
        'lowercollapse0': [float(fit_model.parameters()['drift']['lowercollapse0'])],
        'IC0': [float(fit_model.parameters()['IC']['x00'])],
        'nondec0': [float(fit_model.parameters()['overlay']['nondectime0'])],
        'driftpresent1': [float(fit_model.parameters()['drift']['driftpresent1'])],
        'bound1': [float(fit_model.parameters()['bound']['B1'])],
        'uppercollapse1': [float(fit_model.parameters()['drift']['uppercollapse1'])],
        'lowercollapse1': [float(fit_model.parameters()['drift']['lowercollapse1'])],
        'IC1': [float(fit_model.parameters()['IC']['x01'])],
        'nondec1': [float(fit_model.parameters()['overlay']['nondectime1'])],
        'BIC': [fit_model.fitresult.value()]
    })
    
    if not os.path.exists("model_fits/model_m2c/data_"+data_label):
                       os.makedirs("model_fits/model_m2c/data_"+data_label)
                       
    with open("model_fits/model_m2c/data_"+data_label+"/subj"+str(subj_id)+".txt", "w") as f:
        f.write(repr(fit_model))
    
    return(summary_df)
        

loaded = pd.read_csv("data/E2.csv") 
loaded['present'] = loaded['present'].replace(-1,0)
loaded['condition'] = loaded['condition_is_congruent'].replace(-1,0)

summary_list = []
for name, group in loaded.groupby('subj_id'):
    print(name)
    a= fit_m2c(group.reset_index(),'E2')
    summary_list.append(a)
summary_df = pd.concat(summary_list)
summary_df.to_csv("model_fits/model_m2c/data_E2/all_subjects.csv")

1


Info: Params [ 9.25422412e-01 -1.45443733e-01  1.08086706e-05  6.43905670e-01
  0.00000000e+00  0.00000000e+00  6.71995669e-01  5.97712767e-01
 -2.80918092e-02  5.61409389e-03  6.61843633e-01  5.93135652e-01] gave 147.67377083551852


2


Info: Params [ 0.37150041 -0.19505336  0.29585649  0.72983861 -0.12203764  0.66554657
  1.02548245  1.20492232 -0.02502795 -0.0591806   0.76986226  0.90590027] gave 214.22117685594074


3


...

Info: Params [ 0.90376844 -0.44607176  0.76413932  0.05762099 -1.04566776  0.17655446
  1.97359143  1.83758541 -0.35021728 -0.20318954  0.99523699  0.95111591] gave 234.0970277320043


297


Info: Params [ 0.66610409 -0.04004139  0.05995861  0.46545156 -0.22829037  0.01171413
  0.94500479  1.10187932 -0.16667321 -0.21459351  0.98044222  0.9465472 ] gave 229.81076257929783


298


Info: Params [ 1.10964154 -0.07739868  0.93366302  0.57101656 -0.48837431  0.24019711
  2.44690412  1.82497113  0.02029316  0.10074447  0.73482942  0.54829458] gave 258.3911733188597


## Fit M3: Lower boundary collapses rapidly


In [4]:
class BoundNonlinearCollapseC(Bound):
    name = "Lower boundary collapses nonlinearly"
    required_parameters = ["B0", "lowercollapse0", "collapsetime0",
                          "B1", "lowercollapse1", "collapsetime1"]
    required_conditions = ["condition"]
    def get_bound(self, t, conditions, **kwargs):
        B = [self.B0, self.B1][int(conditions['condition'])]
        lowercollapse = [self.lowercollapse0, self.lowercollapse1][int(conditions['condition'])]
        collapsetime = [self.collapsetime0, self.collapsetime1][int(conditions['condition'])]
        if t<collapsetime:
            return B
        else:
            return max(0.01, B - (t-collapsetime)*lowercollapse/2)
            


class DriftPresentNonlinearCollapseC(ddm.models.Drift):
    name = "Drift depends on target presence and collapses nonlinearly"
    required_parameters = ["driftpresent0","lowercollapse0", "collapsetime0", \
                          "driftpresent1","lowercollapse1", "collapsetime1"] # <-- Parameters we want to include in the model
    required_conditions = ["present", "condition"] # <-- Task parameters ("conditions"). Should be the same name as in the sample.

    # We must always define the get_drift function, which is used to compute the instantaneous value of drift.
    def get_drift(self, t, conditions, **kwargs):
        
        driftpresent = [self.driftpresent0, self.driftpresent1][int(conditions['condition'])]
        lowercollapse = [self.lowercollapse0, self.lowercollapse1][int(conditions['condition'])]
        collapsetime = [self.collapsetime0, self.collapsetime1][int(conditions['condition'])]
        
        if t<collapsetime:
            return conditions['present'] * driftpresent
        else:
            return conditions['present'] * (driftpresent-lowercollapse/2) + \
        (1-conditions['present']) * (0+ lowercollapse/2)


In [6]:
from pyddm import FitResult

lowercollapse0 = Fittable(minval=0, maxval=10)
collapsetime0 = Fittable(minval=0, maxval=5)
lowercollapse1 = Fittable(minval=0, maxval=10)
collapsetime1 = Fittable(minval=0, maxval=5)

m3c_spec = Model(name='Absence drift is zero lower boundary can collapse nonlinearly',
                 drift=DriftPresentNonlinearCollapseC(driftpresent1=Fittable(minval=0, maxval=10),
                                   driftpresent0=Fittable(minval=0, maxval=10),
                                   lowercollapse1=lowercollapse1,
                                   lowercollapse0=lowercollapse0,
                                   collapsetime1=collapsetime1,
                                   collapsetime0=collapsetime0),
                 noise=NoiseConstant(noise=1),
                 IC=ICPointSideBiasRatioC(x00=Fittable(minval=-1, maxval=1),
                                         x01=Fittable(minval=-1, maxval=1)),
                 bound=BoundNonlinearCollapseC(B0=Fittable(minval=0.1, maxval=2.5),
                                    lowercollapse0=lowercollapse0,
                                    collapsetime0=collapsetime0,
                                    B1=Fittable(minval=0.1, maxval=2.5),
                                    lowercollapse1=lowercollapse1,
                                    collapsetime1=collapsetime1),
                 overlay=OverlayChain(overlays=[OverlayNonDecisionC(nondectime0=Fittable(minval=0, maxval=1),
                                                                   nondectime1=Fittable(minval=0,maxval=1)),
                                                OverlayPoissonMixture(pmixturecoef=.02,
                                                                      rate=1)]),
                 dx=.01, dt=.1, T_dur=5)

def fit_m3c(df,data_label):
    
    df = df.reset_index();
    subj_id = df.subj_id[0]
    
    sample = Sample.from_pandas_dataframe(df, rt_column_name="rt", correct_column_name="correct")
    fit_model = fit_adjust_model(sample=sample, model=m3c_spec, verbose=False, lossfunction = LossRobustBIC)
    summary_df = pd.DataFrame.from_dict({
        'subj_id': [subj_id],
        'driftpresent0': [float(fit_model.parameters()['drift']['driftpresent0'])],
        'noise': [float(fit_model.parameters()['noise']['noise'])],
        'bound0': [float(fit_model.parameters()['bound']['B0'])],
        'collapsetime0': [float(fit_model.parameters()['drift']['collapsetime0'])],
        'lowercollapse0': [float(fit_model.parameters()['drift']['lowercollapse0'])],
        'IC0': [float(fit_model.parameters()['IC']['x00'])],
        'nondec0': [float(fit_model.parameters()['overlay']['nondectime0'])],
        'driftpresent1': [float(fit_model.parameters()['drift']['driftpresent1'])],
        'bound1': [float(fit_model.parameters()['bound']['B1'])],
        'collapsetime0': [float(fit_model.parameters()['drift']['collapsetime0'])],
        'lowercollapse1': [float(fit_model.parameters()['drift']['lowercollapse1'])],
        'IC1': [float(fit_model.parameters()['IC']['x01'])],
        'nondec1': [float(fit_model.parameters()['overlay']['nondectime1'])],
        'BIC': [fit_model.fitresult.value()]
    })
    
    if not os.path.exists("model_fits/model_m3c/data_"+data_label):
                       os.makedirs("model_fits/model_m3c/data_"+data_label)
                       
    with open("model_fits/model_m3c/data_"+data_label+"/subj"+str(subj_id)+".txt", "w") as f:
        f.write(repr(fit_model))
    
    return(summary_df)
        

loaded = pd.read_csv("data/E2.csv") 
loaded['present'] = loaded['present'].replace(-1,0)
loaded['condition'] = loaded['condition_is_congruent'].replace(-1,0)

summary_list = []
for name, group in loaded.groupby('subj_id'):
    print(name)
    if not os.path.exists("model_fits/model_m3c/data_E2/subj"+str(name)+".txt"):
        a= fit_m3c(group.reset_index(),'E2')
    else:
        with open("model_fits/model_m3c/data_E2/subj"+str(name)+".txt") as f:
            a = eval(f.read())
    summary_list.append(a)
summary_df = pd.concat(summary_list)
summary_df.to_csv("model_fits/model_m3c/data_E2/all_subjects.csv")

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25


26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50


51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75


76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101


102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129


...

297


Info: Params [ 0.64929344  9.91903955  2.92547542  0.64499014  4.08910878  3.09999884
  0.90276337  1.02167395 -0.17546834 -0.22546919  0.92621424  0.98686135] gave 221.3684330072526


298


Info: Params [ 1.00585992e+00  8.77299308e-01  4.47638799e-01  6.48837702e-01
  2.33604927e+00  2.60935607e+00  1.90850591e+00  1.19743852e+00
 -2.47803397e-03  1.04822999e-01  9.01758525e-01  8.69234323e-01] gave 248.2495654979767


TypeError: cannot concatenate object of type '<class 'pyddm.model.Model'>'; only Series and DataFrame objs are valid

## Fit M4: unequal variance

In [12]:
class NoiseAsymmetric(Noise):
    name = "Noise is higher when a target is present"
    required_parameters = ["noiseabsent"]
    required_conditions = ['present']
    def get_noise(self, t, conditions, **kwargs):
        if conditions['present']==1:
            return 1
        else:
            return self.noiseabsent

In [13]:
m4_spec = Model(name='Basic model drift varies as a function of target presence',
                 drift=DriftPresent(driftabsent=Fittable(minval=-10, maxval=0),
                                     driftpresent=Fittable(minval=0, maxval=10)),
                 noise=NoiseAsymmetric(noiseabsent=Fittable(minval=0.5,maxval=2)),
                 IC=ICPointSideBiasRatio(x0=Fittable(minval=-1, maxval=1)),
                 bound=BoundConstant(B=Fittable(minval=0.1, maxval=2.5)),
                 overlay=OverlayChain(overlays=[OverlayNonDecision(nondectime=Fittable(minval=0, maxval=1)),
                                                OverlayPoissonMixture(pmixturecoef=.02,
                                                                      rate=1)]),
                 dx=.01, dt=.1, T_dur=5)

def fit_m4(df,data_label):
    
    df = df.reset_index();
    subj_id = df.subj_id[0]
    
    sample = Sample.from_pandas_dataframe(df, rt_column_name="rt", correct_column_name="correct")
    fit_model = fit_adjust_model(sample=sample, model=m4_spec, verbose=False, lossfunction = LossRobustBIC)
    summary_df = pd.DataFrame.from_dict({
        'subj_id': [subj_id],
        'driftabsent': [float(fit_model.parameters()['drift']['driftabsent'])],
        'driftpresent': [float(fit_model.parameters()['drift']['driftpresent'])],
        'noiseabsent': [float(fit_model.parameters()['noise']['noiseabsent'])],
        'bound': [float(fit_model.parameters()['bound']['B'])],
        'IC': [float(fit_model.parameters()['IC']['x0'])],
        'nondec': [float(fit_model.parameters()['overlay']['nondectime'])],
        'BIC': [fit_model.fitresult.value()]
    })
    
    if not os.path.exists("model_fits/model_m4/data_"+data_label):
                       os.makedirs("model_fits/model_m4/data_"+data_label)
                       
    with open("model_fits/model_m4/data_"+data_label+"/subj"+str(subj_id)+".txt", "w") as f:
        f.write(repr(fit_model))
    
    return(summary_df)

# loaded = pd.read_csv("data/E2.csv") 
# loaded['present'] = loaded['present'].replace(-1,0)
# summary_list = []
# for name, group in loaded.groupby('subj_id'):
#     print(name)
#     a= fit_m4(group.reset_index(),'E2')
#     summary_list.append(a)
# summary_df = pd.concat(summary_list)
# summary_df.to_csv("model_fits/model_m4/data_E2/all_subjects.csv")

## Fit M5: unequal variance + collapsing boundaries

In [14]:
m5_spec = Model(name='Absence drift is zero boundaries can collapse',
                 drift=DriftPresentCollapse(driftpresent=Fittable(minval=0, maxval=10),
                                   uppercollapse=uppercollapse,
                                   lowercollapse=lowercollapse),
                 noise=NoiseAsymmetric(noiseabsent=Fittable(minval=0.5,maxval=2)),
                 IC=ICPointSideBiasRatio(x0=Fittable(minval=-1, maxval=1)),
                 bound=BoundCollapse(B=Fittable(minval=0.1, maxval=2.5),
                                    uppercollapse=uppercollapse,
                                    lowercollapse=lowercollapse),
                 overlay=OverlayChain(overlays=[OverlayNonDecision(nondectime=Fittable(minval=0, maxval=1)),
                                                OverlayPoissonMixture(pmixturecoef=.02,
                                                                      rate=1)]),
                 dx=.01, dt=.1, T_dur=5)

def fit_m5(df,data_label):
    
    df = df.reset_index();
    subj_id = df.subj_id[0]
    
    sample = Sample.from_pandas_dataframe(df, rt_column_name="rt", correct_column_name="correct")
    fit_model = fit_adjust_model(sample=sample, model=m5_spec, verbose=False, lossfunction = LossRobustBIC)
    summary_df = pd.DataFrame.from_dict({
        'subj_id': [subj_id],
        'driftpresent': [float(fit_model.parameters()['drift']['driftpresent'])],
        'uppercollapse': [float(fit_model.parameters()['drift']['uppercollapse'])],
        'lowercollapse': [float(fit_model.parameters()['drift']['lowercollapse'])],
        'noiseabsent': [float(fit_model.parameters()['noise']['noiseabsent'])],
        'bound': [float(fit_model.parameters()['bound']['B'])],
        'IC': [float(fit_model.parameters()['IC']['x0'])],
        'nondec': [float(fit_model.parameters()['overlay']['nondectime'])],
        'BIC': [fit_model.fitresult.value()]
    })
    
    if not os.path.exists("model_fits/model_m5/data_"+data_label):
                       os.makedirs("model_fits/model_m5/data_"+data_label)
                       
    with open("model_fits/model_m5/data_"+data_label+"/subj"+str(subj_id)+".txt", "w") as f:
        f.write(repr(fit_model))
    
    return(summary_df)
        

# loaded = pd.read_csv("data/E2.csv") 
# loaded['present'] = loaded['present'].replace(-1,0)
# summary_list = []
# for name, group in loaded.groupby('subj_id'):
#     print(name)
#     a= fit_m5(group.reset_index(),'E2')
#     summary_list.append(a)
# summary_df = pd.concat(summary_list)
# summary_df.to_csv("model_fits/model_m5/data_E2/all_subjects.csv")

## Fit M6: unequal variance + nonlinearly collapsing boundary

In [15]:
m6_spec = Model(name='Absence drift is zero lower boundary can collapse nonlinearly',
                 drift=DriftPresentNonlinearCollapse(driftpresent=Fittable(minval=0, maxval=10),
                                   lowercollapse=lowercollapse,
                                   collapsetime=collapsetime),
                 noise=NoiseAsymmetric(noiseabsent=Fittable(minval=0,maxval=3)),
                 IC=ICPointSideBiasRatio(x0=Fittable(minval=-1, maxval=1)),
                 bound=BoundNonlinearCollapse(B=Fittable(minval=0.1, maxval=2.5),
                                    lowercollapse=lowercollapse,
                                    collapsetime=collapsetime),
                 overlay=OverlayChain(overlays=[OverlayNonDecision(nondectime=Fittable(minval=0, maxval=1)),
                                                OverlayPoissonMixture(pmixturecoef=.02,
                                                                      rate=1)]),
                 dx=.01, dt=.1, T_dur=5)

def fit_m6(df,data_label):
    
    df = df.reset_index();
    subj_id = df.subj_id[0]
    
    sample = Sample.from_pandas_dataframe(df, rt_column_name="rt", correct_column_name="correct")
    fit_model = fit_adjust_model(sample=sample, model=m6_spec, verbose=False, lossfunction = LossRobustBIC)
    summary_df = pd.DataFrame.from_dict({
        'subj_id': [subj_id],
        'driftpresent': [float(fit_model.parameters()['drift']['driftpresent'])],
        'collapsetime': [float(fit_model.parameters()['drift']['collapsetime'])],
        'lowercollapse': [float(fit_model.parameters()['drift']['lowercollapse'])],
        'noiseabsent': [float(fit_model.parameters()['noise']['noiseabsent'])],
        'bound': [float(fit_model.parameters()['bound']['B'])],
        'IC': [float(fit_model.parameters()['IC']['x0'])],
        'nondec': [float(fit_model.parameters()['overlay']['nondectime'])],
        'BIC': [fit_model.fitresult.value()]
    })
    
    if not os.path.exists("model_fits/model_m6/data_"+data_label):
                       os.makedirs("model_fits/model_m6/data_"+data_label)
                       
    with open("model_fits/model_m6/data_"+data_label+"/subj"+str(subj_id)+".txt", "w") as f:
        f.write(repr(fit_model))
    
    return(summary_df)
        

# loaded = pd.read_csv("data/E2.csv") 
# loaded['present'] = loaded['present'].replace(-1,0)
# summary_list = []
# for name, group in loaded.groupby('subj_id'):
#     print(name)
#     a= fit_m6(group.reset_index(),'E2')
#     summary_list.append(a)
# summary_df = pd.concat(summary_list)
# summary_df.to_csv("model_fits/model_m6/data_E2/all_subjects.csv")

## Fit M7: basic model with changing t0

In [34]:
from pyddm import OverlayNonDecision, Solution
class OverlayNonDecisionPresent(OverlayNonDecision):
    name = "Separate non-decision time for target presence or absence"
    required_parameters = ["nondectimepresent", "nondectimeabsent"]
    required_conditions = ['present']
    def get_nondecision_time(self, conditions):
        return self.nondectimepresent if conditions['present'] == 1 else self.nondectimeabsent

In [35]:
m7_spec = Model(name='Basic model drift varies as a function of target presence',
                 drift=DriftPresent(driftabsent=Fittable(minval=-10, maxval=0),
                                     driftpresent=Fittable(minval=0, maxval=10)),
                 noise=NoiseConstant(noise=1),
                 IC=ICPointSideBiasRatio(x0=Fittable(minval=-1, maxval=1)),
                 bound=BoundConstant(B=Fittable(minval=0.1, maxval=2.5)),
                 overlay=OverlayChain(overlays=[OverlayNonDecisionPresent(nondectimepresent=Fittable(minval=0, maxval=1),
                                                                         nondectimeabsent = Fittable(minval=0, maxval=1)),
                                                OverlayPoissonMixture(pmixturecoef=.02,
                                                                      rate=1)]),
                 dx=.01, dt=.1, T_dur=5)

def fit_m7(df,data_label):
    
    df = df.reset_index();
    subj_id = df.subj_id[0]
    
    sample = Sample.from_pandas_dataframe(df, rt_column_name="rt", correct_column_name="correct")
    fit_model = fit_adjust_model(sample=sample, model=m7_spec, verbose=False, lossfunction = LossRobustBIC)
    summary_df = pd.DataFrame.from_dict({
        'subj_id': [subj_id],
        'driftabsent': [float(fit_model.parameters()['drift']['driftabsent'])],
        'driftpresent': [float(fit_model.parameters()['drift']['driftpresent'])],
        'noise': [float(fit_model.parameters()['noise']['noise'])],
        'bound': [float(fit_model.parameters()['bound']['B'])],
        'IC': [float(fit_model.parameters()['IC']['x0'])],
        'nondecpresent': [float(fit_model.parameters()['overlay']['nondectimepresent'])],
        'nondecabsent': [float(fit_model.parameters()['overlay']['nondectimeabsent'])],
        'BIC': [fit_model.fitresult.value()]
    })
    
    if not os.path.exists("model_fits/model_m7/data_"+data_label):
                       os.makedirs("model_fits/model_m7/data_"+data_label)
                       
    with open("model_fits/model_m7/data_"+data_label+"/subj"+str(subj_id)+".txt", "w") as f:
        f.write(repr(fit_model))
    
    return(summary_df)

In [36]:
loaded = pd.read_csv("data/E2.csv") 
loaded['present'] = loaded['present'].replace(-1,0)
summary_list = []
for name, group in loaded.groupby('subj_id'):
    print(name)
    a= fit_m7(group.reset_index(),'E2')
    summary_list.append(a)
summary_df = pd.concat(summary_list)
summary_df.to_csv("model_fits/model_m7/data_E2/all_subjects.csv")

1


Info: Params [ 0.         -1.94030983  0.69793247  0.50168669  0.6095783   0.57921409] gave 101.59308630397221


2


Info: Params [ 0.         -1.38401023  0.90966948  0.2799799   0.83639137  0.98410181] gave 169.46592794966512


3


...

Info: Params [ 0.03466987 -1.39167683  1.38503741  0.163835    0.91387357  0.95844566] gave 205.93685838462426


297


Info: Params [ 0.37177865 -0.89564915  0.97392639  0.02038952  0.92802343  0.9292813 ] gave 194.8945007553816


298


Info: Params [ 0.67863041 -0.31107945  1.31457587  0.12337219  0.90538316  0.9548102 ] gave 239.98837126890638


In [ ]:
models = [0, fit_m1, fit_m2, fit_m3, fit_m4, fit_m5, fit_m6]

for i_data in list(range(3,5)):
    loaded = pd.read_csv("data/E"+str(i_data)+".csv") 
    loaded['present'] = loaded['present'].replace(-1,0)
    for i_model in list(range(1,7)):
        summary_list = []
        for name, group in loaded.groupby('subj_id'):
            print(name)
            fit_result = models[i_model](group.reset_index(),'E2')
            summary_list.append(fit_result)
        summary_df = pd.concat(summary_list)
        if not os.path.exists("model_fits/model_m"+str(i_model)+"/data_E"+str(i_data)):
                   os.makedirs("model_fits/model_m"+str(i_model)+"/data_E"+str(i_data))
        summary_df.to_csv("model_fits/model_m"+str(i_model)+"/data_E"+str(i_data)+"/all_subjects.csv")

1


Info: Params [ 0.40251789 -1.34956371  1.38440093  0.23084832  0.92918123] gave 210.11483117741562


2


Info: Params [ 1.71068887 -1.85767097  0.56922261  0.07482092  0.43234318] gave 23.593523417861512


3


...

44


Info: Params [1.06024449 0.         1.38749999 2.49378261 0.17927082 0.93920587] gave 190.57960495421236


45


Info: Params [0.86587273 0.         1.05263163 2.49562784 0.11272983 0.99195629] gave 231.273596991056


46


In [16]:
from pyddm import FitResult
from pyddm import Fittable, Fitted
import pyddm.plot


def plotSubj(subj_id, model_fit, exp):
    
    with open("model_fits/model_"+model_fit+"/data_"+exp+"/subj"+str(subj_id)+".txt", "r") as f:
        model = eval(f.read())
        
    data_loaded = pd.read_csv("data/"+exp+".csv") 

    pyddm.plot.model_gui(model=model, sample = Sample.from_pandas_dataframe(data_loaded[data_loaded.subj_id==subj_id], rt_column_name="rt", correct_column_name="correct"))


In [46]:
plotSubj(8,'m3','E2')